## Revised Linear Layer Architecture without VIB

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import transforms
from tensorboardX import SummaryWriter
#from utils import cuda
import pdb
import time
from numbers import Number
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import sklearn.metrics
import argparse
from torch.utils.data import Dataset, DataLoader

### neural networks architecture

In [2]:
class ToyNet(nn.Module):
    '''
    Construct a MLP that is used to train the model 
    param[in]: X_train, output_features
    param[out]: output 
    
    Note: initialize the weight with a self-defined method
    '''

    def __init__(self, output_features=8):
        super(ToyNet, self).__init__()
        self.encode = nn.Sequential(
            nn.Linear(40, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 1024),
            nn.ReLU(True),
            nn.Linear(1024, output_features))
        #self.optim = optim.Adam(self.toynet.parameters(),lr=self.lr,betas=(0.5,0.999))
        #self.scheduler = lr_scheduler.ExponentialLR(self.optim,gamma=0.97
    def forward(self, X_train):
        output=self.encode(X_train)
        #prediction = F.softmax(output,dim=1).max(1)[1]
        
        #print(prediction)
        return output
    def weight_init(self):
        for m in self._modules:
            xavier_init(self._modules[m])
            
def xavier_init(ms):
    """
    Xavier initialization
    """
    for m in ms :
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
            nn.init.xavier_uniform(m.weight,gain=nn.init.calculate_gain('relu'))
            m.bias.data.zero_()

In [3]:
def cuda(tensor, is_cuda):
    if is_cuda : return tensor.cuda()
    else : return tensor

In [13]:
params={
    'epoch':100,
    'batch_size':32,
    'lr':0.009,
    'train_dataset_percentage':0.05
}

In [14]:
class CustomDataset(Dataset):
    """
    construct dataset from numpy and split it 
    
    """
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)
    
    
X = joblib.load('./joblib_features/X.joblib')
y = joblib.load('./joblib_features/y.joblib')
full_dataset = CustomDataset(X, y)
train_size = int(0.6 * len(full_dataset))
valid_size = int(0.2 * len(full_dataset))
test_size = len(full_dataset) - valid_size-train_size
batch_size = params['batch_size']
#train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, 
                                                            #[train_size, test_size])
### split dataset into training, validation and test                     
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
    full_dataset, (train_size, valid_size, test_size), generator=torch.Generator().manual_seed(42))
train_dataset_inuse_size=int(params['train_dataset_percentage']* len(full_dataset))
train_dataset_unused_size=len(train_dataset)- train_dataset_inuse_size
train_dataset_inuse,train_dataset_unused=torch.utils.data.random_split(
    train_dataset, (train_dataset_inuse_size,train_dataset_unused_size), generator=torch.Generator().manual_seed(42))
train_dataloader = DataLoader(train_dataset_inuse, batch_size=batch_size, shuffle=True, num_workers=0)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
my_dataloader = {'train': train_dataloader , 'validate':valid_dataloader, 'test': test_dataloader}
print('train inuse,train, percentage of whole ,valid,test')
print(train_dataset_inuse_size,train_size, train_dataset_inuse_size/len(full_dataset),valid_size,test_size)


train inuse,train, percentage of whole ,valid,test
262 3151 0.049885757806549885 1050 1051


In [15]:
class Solver(object):
#train the model

    def __init__(self, params):
        """
        initialization of a Solver object
        
        """
        self.cuda =torch.cuda.is_available ()
        self.epoch = params['epoch']
        self.batch_size = params['batch_size']
        self.lr = params['lr']
        self.toynet = cuda(ToyNet(), self.cuda)
        self.toynet.weight_init()
        self.optim = optim.Adam(self.toynet.parameters(),
                                lr=self.lr,
                                betas=(0.5,0.999))
        self.criterion = nn.CrossEntropyLoss()
    
    def train(self):
        baseline_train = {"Accuracy":[],"F1_Score":[]}
        baseline_valid = {"Accuracy":[],"F1_Score":[]}
        for epc in range(self.epoch):  # loop over the dataset multiple times
            self.toynet.train('True')  # training neural networks mode
            running_loss = 0.0
            y_real=torch.randn([0])
            y_hat=torch.randn([0])
            for i, data in enumerate(train_dataloader):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data

                # zero the parameter gradients
                self.optim.zero_grad()

                # forward + backward + optimize
                outputs = self.toynet.forward(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optim.step()
                
                prediction = F.softmax(outputs,dim=1).max(1)[1]
                y_real=torch.cat([y_real,labels],dim=0)
                y_hat=torch.cat([y_hat,prediction],dim=0)
                accuracy = torch.eq(prediction,labels).float().mean()
                avg_accuracy = Variable(cuda(torch.zeros(accuracy.size()), self.cuda))

                # print statistics
                running_loss += loss.item()
                if i % self.batch_size == 0:    # print every 30 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                            (epc + 1, i + 1, running_loss / self.batch_size))
                    print('acc:{:.4f} '
                            .format(accuracy.item(), end=' '))
                    print('err:{:.4f} '
                            .format(1-accuracy.item()))
                    running_loss = 0.0
            baseline_train["Accuracy"].append(float("{:.2f}".format(accuracy.item())))
            f1score = sklearn.metrics.f1_score(y_real, y_hat,labels=None,pos_label=1, average='weighted',sample_weight=None)
            baseline_train["F1_Score"].append(float("{:.2f}".format(f1score.item())))
            ## validation set at each epoch
            temp_accuracy,temp_f1score = self.test()
            ##input accuracy and f1-score of validation dataset into ano
            baseline_valid["F1_Score"].append(float("{:.2f}".format(temp_f1score)))
            baseline_valid["Accuracy"].append(float("{:.2f}".format(temp_accuracy)))
        print(baseline_train)
        print(baseline_valid)
        print('Finished Training',(epc+1))
        return baseline_train, baseline_valid
    
    def test(self):
        """
        Testing over a dataset
        """
        self.toynet.train('False')  # evaluation mode      
        loss, correct, total_num = 0,0,0
        
        y_real=torch.randn([0])
        y_hat=torch.randn([0])
        
        for i, data in enumerate(valid_dataloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            outputs = self.toynet.forward(inputs)
            #loss
            total_num += labels.shape[0]
            loss += self.criterion(outputs,labels)
            prediction = F.softmax(outputs,dim=1).max(1)[1]
            y_real=torch.cat([y_real,labels],dim=0)
            y_hat=torch.cat([y_hat,prediction],dim=0)
            correct += torch.eq(prediction,labels).float().sum()
            avg_correct = Variable(cuda(torch.zeros(correct.size()), self.cuda))
        accuracy = correct/total_num
        f1score = sklearn.metrics.f1_score(y_real, y_hat,labels=None,pos_label=1, average='weighted',sample_weight=None)
        avg_accuracy = avg_correct/total_num
            
        print('[TEST RESULT]')
        print('acc:{:.4f} '
                .format(accuracy.item(),end=' '))
        print('err:{:.4f}'
                .format(1-accuracy.item()))
        print(classification_report(y_real,y_hat))
        self.toynet.train('True')
        return accuracy.item(),f1score.item()
    

In [16]:
net=Solver(params)
baseline_train, baseline_valid=net.train()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


[1,     1] loss: 0.867
acc:0.0625 
err:0.9375 
[TEST RESULT]
acc:0.1762 
err:0.8238
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       115
         1.0       0.16      0.84      0.27        63
         2.0       0.00      0.00      0.00       149
         3.0       0.00      0.00      0.00       159
         4.0       0.19      0.73      0.30       180
         5.0       0.00      0.00      0.00       157
         6.0       0.00      0.00      0.00       109
         7.0       0.00      0.00      0.00       118

    accuracy                           0.18      1050
   macro avg       0.04      0.20      0.07      1050
weighted avg       0.04      0.18      0.07      1050

[2,     1] loss: 0.626
acc:0.2500 
err:0.7500 
[TEST RESULT]
acc:0.3733 
err:0.6267
              precision    recall  f1-score   support

         0.0       0.31      0.82      0.45       115
         1.0       0.00      0.00      0.00        63
         2.0      

[14,     1] loss: 0.095
acc:0.5000 
err:0.5000 
[TEST RESULT]
acc:0.2952 
err:0.7048
              precision    recall  f1-score   support

         0.0       0.23      0.80      0.36       115
         1.0       0.14      0.84      0.24        63
         2.0       0.47      0.32      0.38       149
         3.0       0.88      0.14      0.24       159
         4.0       0.65      0.26      0.37       180
         5.0       0.81      0.11      0.19       157
         6.0       0.00      0.00      0.00       109
         7.0       0.60      0.27      0.37       118

    accuracy                           0.30      1050
   macro avg       0.47      0.34      0.27      1050
weighted avg       0.53      0.30      0.28      1050

[15,     1] loss: 0.098
acc:0.4375 
err:0.5625 
[TEST RESULT]
acc:0.4524 
err:0.5476
              precision    recall  f1-score   support

         0.0       1.00      0.75      0.86       115
         1.0       0.16      0.21      0.18        63
         2.0    

[27,     1] loss: 0.017
acc:0.8438 
err:0.1562 
[TEST RESULT]
acc:0.2400 
err:0.7600
              precision    recall  f1-score   support

         0.0       0.15      0.98      0.26       115
         1.0       0.62      0.13      0.21        63
         2.0       0.36      0.23      0.29       149
         3.0       1.00      0.01      0.01       159
         4.0       1.00      0.11      0.20       180
         5.0       0.48      0.39      0.43       157
         6.0       0.00      0.00      0.00       109
         7.0       0.32      0.11      0.16       118

    accuracy                           0.24      1050
   macro avg       0.49      0.25      0.20      1050
weighted avg       0.53      0.24      0.20      1050

[28,     1] loss: 0.351
acc:0.3750 
err:0.6250 
[TEST RESULT]
acc:0.4019 
err:0.5981
              precision    recall  f1-score   support

         0.0       0.98      0.70      0.82       115
         1.0       0.25      0.63      0.36        63
         2.0    

[TEST RESULT]
acc:0.5695 
err:0.4305
              precision    recall  f1-score   support

         0.0       0.54      0.84      0.66       115
         1.0       0.23      0.97      0.38        63
         2.0       0.80      0.44      0.57       149
         3.0       0.88      0.43      0.58       159
         4.0       0.81      0.54      0.65       180
         5.0       0.59      0.56      0.58       157
         6.0       0.54      0.54      0.54       109
         7.0       0.87      0.51      0.64       118

    accuracy                           0.57      1050
   macro avg       0.66      0.61      0.57      1050
weighted avg       0.70      0.57      0.59      1050

[40,     1] loss: 0.039
acc:0.5938 
err:0.4062 
[TEST RESULT]
acc:0.5714 
err:0.4286
              precision    recall  f1-score   support

         0.0       0.92      0.76      0.83       115
         1.0       0.41      0.14      0.21        63
         2.0       0.71      0.51      0.59       149
         3

[TEST RESULT]
acc:0.5857 
err:0.4143
              precision    recall  f1-score   support

         0.0       0.90      0.77      0.83       115
         1.0       0.27      0.78      0.40        63
         2.0       0.73      0.43      0.54       149
         3.0       0.79      0.55      0.65       159
         4.0       0.79      0.59      0.68       180
         5.0       0.46      0.72      0.56       157
         6.0       0.51      0.50      0.50       109
         7.0       0.60      0.44      0.51       118

    accuracy                           0.59      1050
   macro avg       0.63      0.60      0.58      1050
weighted avg       0.66      0.59      0.60      1050

[52,     1] loss: 0.017
acc:0.7812 
err:0.2188 
[TEST RESULT]
acc:0.5552 
err:0.4448
              precision    recall  f1-score   support

         0.0       0.77      0.81      0.79       115
         1.0       0.74      0.22      0.34        63
         2.0       0.79      0.44      0.57       149
         3

[TEST RESULT]
acc:0.5924 
err:0.4076
              precision    recall  f1-score   support

         0.0       0.62      0.85      0.72       115
         1.0       0.44      0.29      0.35        63
         2.0       0.57      0.51      0.54       149
         3.0       0.45      0.72      0.55       159
         4.0       0.81      0.53      0.64       180
         5.0       0.64      0.52      0.58       157
         6.0       0.66      0.60      0.63       109
         7.0       0.61      0.62      0.61       118

    accuracy                           0.59      1050
   macro avg       0.60      0.58      0.58      1050
weighted avg       0.62      0.59      0.59      1050

[64,     1] loss: 0.013
acc:0.8125 
err:0.1875 
[TEST RESULT]
acc:0.6067 
err:0.3933
              precision    recall  f1-score   support

         0.0       0.82      0.78      0.80       115
         1.0       0.58      0.29      0.38        63
         2.0       0.62      0.56      0.59       149
         3

[TEST RESULT]
acc:0.6105 
err:0.3895
              precision    recall  f1-score   support

         0.0       0.52      0.85      0.64       115
         1.0       0.51      0.35      0.42        63
         2.0       0.58      0.56      0.57       149
         3.0       0.68      0.53      0.60       159
         4.0       0.74      0.64      0.68       180
         5.0       0.57      0.62      0.59       157
         6.0       0.54      0.72      0.62       109
         7.0       0.81      0.53      0.64       118

    accuracy                           0.61      1050
   macro avg       0.62      0.60      0.59      1050
weighted avg       0.63      0.61      0.61      1050

[76,     1] loss: 0.021
acc:0.7188 
err:0.2812 
[TEST RESULT]
acc:0.5190 
err:0.4810
              precision    recall  f1-score   support

         0.0       0.51      0.83      0.63       115
         1.0       0.28      0.43      0.34        63
         2.0       0.67      0.50      0.57       149
         3

[TEST RESULT]
acc:0.5886 
err:0.4114
              precision    recall  f1-score   support

         0.0       0.70      0.83      0.76       115
         1.0       0.40      0.30      0.35        63
         2.0       0.43      0.72      0.53       149
         3.0       0.51      0.70      0.59       159
         4.0       0.71      0.51      0.59       180
         5.0       0.71      0.52      0.60       157
         6.0       0.75      0.33      0.46       109
         7.0       0.72      0.64      0.68       118

    accuracy                           0.59      1050
   macro avg       0.62      0.57      0.57      1050
weighted avg       0.62      0.59      0.58      1050

[88,     1] loss: 0.040
acc:0.6875 
err:0.3125 
[TEST RESULT]
acc:0.5695 
err:0.4305
              precision    recall  f1-score   support

         0.0       0.66      0.83      0.74       115
         1.0       0.43      0.24      0.31        63
         2.0       0.55      0.58      0.56       149
         3

[TEST RESULT]
acc:0.5743 
err:0.4257
              precision    recall  f1-score   support

         0.0       0.71      0.80      0.75       115
         1.0       0.43      0.19      0.26        63
         2.0       0.47      0.62      0.54       149
         3.0       0.56      0.64      0.60       159
         4.0       0.60      0.59      0.59       180
         5.0       0.49      0.59      0.54       157
         6.0       0.67      0.60      0.63       109
         7.0       0.74      0.36      0.48       118

    accuracy                           0.57      1050
   macro avg       0.58      0.55      0.55      1050
weighted avg       0.58      0.57      0.57      1050

[100,     1] loss: 0.016
acc:0.8750 
err:0.1250 
[TEST RESULT]
acc:0.6048 
err:0.3952
              precision    recall  f1-score   support

         0.0       0.86      0.77      0.82       115
         1.0       0.39      0.52      0.45        63
         2.0       0.53      0.58      0.56       149
         

In [17]:
import json
import pathlib

In [18]:
working_dir_path = pathlib.Path().absolute()
SAVE_DIR_PATH = str(working_dir_path) + '/Dictionaries/baseline'
fileName =str(params['train_dataset_percentage'])+'baseline_train'

In [19]:
def writeToJSONFile(path, fileName, data):
    filePathNameWExt =  path + '/' + fileName + '.json'
    with open(filePathNameWExt, 'w') as fp:
        json.dump(data, fp)

# Example
data = baseline_train

writeToJSONFile(SAVE_DIR_PATH,fileName,data)


In [20]:
fileName =str(params['train_dataset_percentage'])+'baseline_valid'

In [21]:
def writeToJSONFile(path, fileName, data):
    filePathNameWExt =  path + '/' + fileName + '.json'
    with open(filePathNameWExt, 'w') as fp:
        json.dump(data, fp)

# Example
data = baseline_valid

writeToJSONFile(SAVE_DIR_PATH,fileName,data)

In [22]:
with open(SAVE_DIR_PATH+'/'+fileName+'.json') as js:
    data = json.load(js)

In [23]:
data

{'Accuracy': [0.18,
  0.37,
  0.46,
  0.49,
  0.45,
  0.41,
  0.5,
  0.51,
  0.48,
  0.46,
  0.51,
  0.41,
  0.46,
  0.3,
  0.45,
  0.54,
  0.42,
  0.51,
  0.55,
  0.47,
  0.59,
  0.57,
  0.59,
  0.56,
  0.6,
  0.62,
  0.24,
  0.4,
  0.47,
  0.48,
  0.5,
  0.54,
  0.56,
  0.56,
  0.53,
  0.45,
  0.53,
  0.57,
  0.57,
  0.57,
  0.58,
  0.57,
  0.6,
  0.58,
  0.61,
  0.61,
  0.6,
  0.59,
  0.6,
  0.62,
  0.59,
  0.56,
  0.58,
  0.6,
  0.61,
  0.61,
  0.62,
  0.62,
  0.62,
  0.64,
  0.59,
  0.59,
  0.59,
  0.61,
  0.61,
  0.6,
  0.63,
  0.62,
  0.63,
  0.63,
  0.6,
  0.54,
  0.56,
  0.5,
  0.61,
  0.52,
  0.6,
  0.6,
  0.6,
  0.61,
  0.59,
  0.61,
  0.63,
  0.59,
  0.57,
  0.5,
  0.59,
  0.57,
  0.62,
  0.63,
  0.58,
  0.63,
  0.6,
  0.6,
  0.52,
  0.59,
  0.6,
  0.54,
  0.57,
  0.6],
 'F1_Score': [0.07,
  0.35,
  0.45,
  0.5,
  0.47,
  0.41,
  0.52,
  0.52,
  0.52,
  0.46,
  0.53,
  0.38,
  0.45,
  0.28,
  0.43,
  0.53,
  0.45,
  0.54,
  0.56,
  0.45,
  0.61,
  0.58,
  0.59,
  0.56,
  0.

In [ ]:
baseline_train['Accuracy']

In [ ]:
dict = {'name': nme, 
        'baseline_train_acc':baseline_train['Accuracy'] ,
        'baseline_valid_acc':baseline_valid['Accuracy'],
        'VIB_train_acc:      'score': scr}  

In [ ]:
nme=['baseline_train_acc','baseline_valid_acc','VIB_train_acc','VIB_test_acc']

In [ ]:
df_baseline_train = pd.DataFrame(baseline_train) 
df_baseline_valid =
df_baseline_train

In [ ]:
baseline_train={'a':1,'b':[1,2]}
def myfunc(baseline_train):
    return baseline_train


In [ ]:
with open('res.json', 'w') as jf:
    json.dump(dic_res['base_train'], jf)

In [ ]:
vib_acc = [0.23, .30, .4]

In [ ]:
with open('res.json', 'r') as jf:
    res=json.load(jf)
res